In [1]:
import cv2
import mediapipe as mp
import time
import math
 
class handDetector():
    def __init__(self,
    static_image_mode=False,
    max_num_hands=2,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5):

        self.mode = static_image_mode
        self.maxHands = max_num_hands
        self.detectionCon = min_detection_confidence
        self.trackCon = min_tracking_confidence
        self.modelComplex = model_complexity
        
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
        self.modelComplex, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]
 
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
    
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
    
            if draw:
                cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20),
            (bbox[2] + 20, bbox[3] + 20), (0, 255, 0), 2)
        
        return self.lmList, bbox
    
    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
        # 4 Fingers
        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        return fingers
    
    def findDistance(self, p1, p2, img, draw=True):
    
        x1, y1 = self.lmList[p1][1], self.lmList[p1][2]
        x2, y2 = self.lmList[p2][1], self.lmList[p2][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        
        if draw:
            cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        
        length = math.hypot(x2 - x1, y2 - y1)
        return length, img, [x1, y1, x2, y2, cx, cy]
    


In [2]:
def codeText(code):

    if(code==16):
        return "Take off"
    elif(code==4):
        return "Do a frontal flip"
    elif(code==28):
        return "Pixelate faces"
    elif(code==24):
        return "Take a picture"
    elif(code==31):
        return "Land"
    elif(code==8):
        return "360º"
    elif(code==12):
        return "Battery level question"

In [3]:
pTime = 0
cap = cv2.VideoCapture(0)
cap.set(3, 300)
cap.set(4, 300)
detector = handDetector()
fingersT=[]
fingers_20=[0,0,0,0,0]
counter=0
action="none"


In [4]:

while True:
    try:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList = detector.findPosition(img)
        fingers=detector.fingersUp()##idea, use this function for gestures(numbers with the hand)
        if(len(fingers)>0):

            if(counter<20):
                counter= counter + 1
                bin_finger=str(fingers[0])+str(fingers[1])+str(fingers[2])+str(fingers[3])+str(fingers[4])
                dec_finger=int(str(bin_finger), 2)
                fingersT.append(dec_finger)
            else:
                if(len(fingersT)>1):
                    fingers_20= max(set(fingersT),key=fingersT.count)
                    action=codeText(fingers_20)
                fingersT=[]
                counter=0

            #print(fingers)
        #if len(lmList) >0:
        #    print(lmList)
    except Exception as e:
        #print(f'exc: {e}')
        pass

    if(action != "none"):
        cv2.line(img, (0, 15), (300, 15), (255, 255, 255), 30)
        cv2.putText(img, action , (10, 18), cv2.FONT_HERSHEY_PLAIN, 1,(40, 40, 40), 2)
    
    
    cv2.imshow("Image", img)
    key = cv2.waitKey(1) & 0xFF    
    
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()